# Ejercicio Covid-19

In [1]:
import os
import numpy as np
import pandas as pd

#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore2 = date_daybefore.strftime('%d/%m')

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_daybefore_large = date_daybefore.strftime('%Y-%m-%d')
date_today_large

'2020-04-21'

## Traer los datos de Datadista

In [2]:
data_ccaa_confirmed_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv", 
                                             sep = ',', encoding="utf-8")

data_ccaa_deaths_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv", 
                                          sep = ',', encoding="utf-8")

data_ccaa_recovered_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv", 
                                             sep = ',', encoding="utf-8")

#Comprobar si están subidos los últimos datos, si no sale error es que hay datos.
data_ccaa_recovered_timeseries[date_today_large]

0      3411
1      1182
2       616
3      1005
4       878
5       646
6      4242
7      5429
8     14446
9        73
10     4719
11      921
12     1588
13    32277
14       46
15      702
16     1190
17     7277
18     1866
19    82514
Name: 2020-04-21, dtype: int64

## RESUMEN: Resumen datos globales

In [3]:
# Función para obtener datos del mundo    
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual

def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"


# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)


# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)

'''
# Para España actual(dataset de datadista)
data_confirmed_spain_actual = int(data_ccaa_confirmed_timeseries[date_today_large].tail(1))
data_recovered_spain_actual = int(data_ccaa_recovered_timeseries[date_today_large].tail(1))
data_deaths_spain_actual = int(data_ccaa_deaths_timeseries[date_today_large].tail(1))
'''

'''
# Para España nuevos casos 24h(dataset de datadista)
data_confirmed_spain_newcases = int(data_ccaa_confirmed_timeseries[date_today_large].tail(1)) - int(data_ccaa_confirmed_timeseries[date_daybefore_large].tail(1))
data_recovered_spain_newcases = int(data_ccaa_recovered_timeseries[date_today_large].tail(1)) - int(data_ccaa_recovered_timeseries[date_daybefore_large].tail(1))
data_deaths_spain_newcases = int(data_ccaa_deaths_timeseries[date_today_large].tail(1)) - int(data_ccaa_deaths_timeseries[date_daybefore_large].tail(1))
'''


# Creamos dataframes

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_recovered_spain_actual, 
                                    'Fallecidos': data_deaths_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])


data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('data_world_resume.csv', sep = ';')
print(data_resume_final)

Confirmados  Curados  Fallecidos
Casos totales en el mundo           2472258   645738      169985
Nuevos casos en el mundo (24h)        70880    21835        4942
Casos totales en España              200210    80587       20852
Nuevos casos en España (24h)           1536     3230         399


## MUNDO: Preparación general de datos por países

In [4]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-16,83403.0,147091.0,137698.0,77995.0,168941.0,10613.0,6896.0,184948.0,667592.0,104145.0
2020-04-17,83760.0,149130.0,141397.0,79494.0,172434.0,10635.0,6937.0,190839.0,699706.0,109769.0
2020-04-18,83787.0,149149.0,143342.0,80868.0,175925.0,10653.0,7036.0,191726.0,732197.0,115314.0
2020-04-19,83805.0,154097.0,145184.0,82211.0,178972.0,10661.0,7078.0,198674.0,759086.0,121172.0
2020-04-20,83817.0,156480.0,147065.0,83505.0,181228.0,10674.0,7156.0,200210.0,784326.0,125856.0


## MUNDO: Confirmados, curados y fallecidos

In [5]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together_final = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together_final.to_csv('data_world_actual.csv', sep = ';')
data_together_final

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,83817.0,77745.0,4636.0
France,156480.0,38036.0,20292.0
Germany,147065.0,91500.0,4862.0
Italy,181228.0,48877.0,24114.0
Spain,200210.0,80587.0,20852.0
US,784326.0,72329.0,42094.0
United Kingdom,125856.0,446.0,16550.0


## MUNDO: Confirmados, curados y fallecidos por habitantes de población

In [6]:
data_together

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,83817.0,156480.0,147065.0,181228.0,200210.0,784326.0,125856.0
Curados,77745.0,38036.0,91500.0,48877.0,80587.0,72329.0,446.0
Fallecidos,4636.0,20292.0,4862.0,24114.0,20852.0,42094.0,16550.0


In [7]:
pob_wuhan = 11895000
pob_china = 1393000000
pob_france = 67028048
pob_germany = 83019200
pob_italy = 60541000
pob_spain = 46940000
pob_US = 328200000
pob_UK = 66650000

# Por cada millón de habitantes hay estos casos...
data_together_den = data_together.copy()
data_together_den['China'] = (data_together_den['China'] / pob_china)*1000000 # Por cada caso infectado hay 142 que no lo están
data_together_den['France'] = (data_together_den['France'] / pob_france)*1000000 # Por cada caso infectado hay 455 que no lo están
data_together_den['Germany'] = (data_together_den['Germany'] / pob_germany)*1000000  # Por cada caso infectado hay 455 que no lo están
data_together_den['Italy'] = (data_together_den['Italy'] / pob_italy)*1000000
data_together_den['Spain'] = (data_together_den['Spain'] / pob_spain)*1000000
data_together_den['US'] = (data_together_den['US'] / pob_US)*1000000
data_together_den['United Kingdom'] = (data_together_den['United Kingdom'] / pob_UK)*1000000
data_together_den = data_together_den.round(0)
# Exporto CSV
data_together_den.to_csv('data_world_actual_den.csv', sep = ';')
data_together_den

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,60.0,2335.0,1771.0,2993.0,4265.0,2390.0,1888.0
Curados,56.0,567.0,1102.0,807.0,1717.0,220.0,7.0
Fallecidos,3.0,303.0,59.0,398.0,444.0,128.0,248.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [8]:
# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,61)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('data_world_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
56,NaN,81102.0,NaN,NaN,181228.0,10635.0,NaN,NaN,NaN,NaN
57,NaN,81156.0,NaN,NaN,NaN,10653.0,NaN,NaN,NaN,NaN
58,NaN,81250.0,NaN,NaN,NaN,10661.0,NaN,NaN,NaN,NaN
59,NaN,81305.0,NaN,NaN,NaN,10674.0,NaN,NaN,NaN,NaN
60,NaN,81435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [9]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('data_world_timeline.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-16,83403.0,147091.0,137698.0,77995.0,168941.0,10613.0,6896.0,184948.0,667592.0,104145.0
2020-04-17,83760.0,149130.0,141397.0,79494.0,172434.0,10635.0,6937.0,190839.0,699706.0,109769.0
2020-04-18,83787.0,149149.0,143342.0,80868.0,175925.0,10653.0,7036.0,191726.0,732197.0,115314.0
2020-04-19,83805.0,154097.0,145184.0,82211.0,178972.0,10661.0,7078.0,198674.0,759086.0,121172.0
2020-04-20,83817.0,156480.0,147065.0,83505.0,181228.0,10674.0,7156.0,200210.0,784326.0,125856.0


## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [10]:
path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# Evolución últimos 20 días de neuvos casos


def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])

#Juntar dataframes
data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,...,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20
Fallecidos (24h) en España,1,78,62,94,53,191,90,207,213,332,...,525,603,547,300,652,607,687,41,410,399
Fallecidos (24h) en Italia,0,439,175,368,349,345,475,427,627,793,...,619,431,566,602,578,525,575,482,433,454
Fallecidos (24h) en Francia,0,31,12,0,58,0,0,95,207,112,...,636,561,574,762,1440,753,762,642,399,548
Fallecidos (24h) en Alemania,0,4,2,2,6,7,4,16,23,17,...,-31,286,172,100,510,248,300,107,127,276


## ESPAÑA: Evolución temporal casos confirmados

In [11]:
# Preparar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries.drop(columns=['cod_ine'])
data_ccaa_confirmed_timeseries_final.rename(columns = {'CCAA': 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.set_index('Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.iloc[0:19]

# Exportar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final.to_csv('data_ccaa_evolution_confirmed.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final

,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,...,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,1,6,8,12,12,13,13,12,21,27,...,10006,10187,10192,10595,10807,11053,11020,11205,11323,11447
Aragón,0,1,1,0,0,0,0,1,6,11,...,4070,4187,4245,4338,4566,4664,4761,4831,4886,4938
Asturias,0,0,0,1,1,1,2,5,5,5,...,1892,1958,2051,2096,2170,2230,2272,2298,2348,2365
Baleares,1,1,2,2,2,2,5,6,6,6,...,1534,1550,1571,1606,1637,1668,1748,1773,1788,1808
Canarias,6,6,6,7,7,7,7,8,11,18,...,1918,1944,1958,1975,1988,2009,2035,2047,2067,2085
Cantabria,0,0,0,1,10,10,10,10,10,10,...,1752,1777,1796,1823,1845,1884,1990,2050,2083,2123
Castilla-La Mancha,0,0,0,1,3,7,12,13,15,15,...,13698,14054,14329,14680,15151,15997,16349,16625,16796,17045
Castilla y León,0,2,2,3,3,8,11,11,14,14,...,12118,12628,13180,13697,14380,14903,15293,15621,15857,16259
Cataluña,2,3,5,6,15,15,15,24,24,24,...,34027,34726,35197,36505,37354,38316,39943,40600,41676,43112


## ESPAÑA: Resúmenes por comunidades autónomas

In [12]:
# SOLO HACER CUANDO LOS DATOS EN DATADISTA ESTÉN ACTUALIZADOS, SI NO, NO FUNCIONA

# Confirmados por CCAA
data_ccaa_confirmed_today = data_ccaa_confirmed_timeseries[['CCAA', date_today_large]]
data_ccaa_confirmed_today.rename(columns = {date_today_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_today = data_ccaa_confirmed_today.set_index('Comunidades Autónomas')
data_ccaa_confirmed_today = data_ccaa_confirmed_today.copy().iloc[0:19]

# New cases
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_confirmed_daybefore.rename(columns = {date_daybefore_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_daybefore.set_index('Comunidades Autónomas')
data_ccaa_confirmed_newcases = data_ccaa_confirmed_today - data_ccaa_confirmed_daybefore
data_ccaa_confirmed_newcases = data_ccaa_confirmed_newcases.copy().iloc[0:19]
data_ccaa_confirmed_newcases.rename(columns = {'Confirmados': 'Confirmados (24h)'}, inplace = True)


# Fallecidos por CCAA
data_ccaa_deaths_today = data_ccaa_deaths_timeseries[['CCAA', date_today_large]]
data_ccaa_deaths_today.rename(columns = {date_today_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_today = data_ccaa_deaths_today.set_index('Comunidades Autónomas')
data_ccaa_deaths_today = data_ccaa_deaths_today.copy().iloc[0:19]

# New cases
data_ccaa_deaths_daybefore = data_ccaa_deaths_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_deaths_daybefore.rename(columns = {date_daybefore_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_daybefore = data_ccaa_deaths_daybefore.set_index('Comunidades Autónomas')
data_ccaa_deaths_newcases = data_ccaa_deaths_today - data_ccaa_deaths_daybefore
data_ccaa_deaths_newcases = data_ccaa_deaths_newcases.copy().iloc[0:19]
data_ccaa_deaths_newcases.rename(columns = {'Fallecidos': 'Fallecidos (24h)'}, inplace = True)


# Recuperados por CCAA
data_ccaa_recovered_today = data_ccaa_recovered_timeseries[['CCAA', date_today_large]]
data_ccaa_recovered_today.rename(columns = {date_today_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_today = data_ccaa_recovered_today.set_index('Comunidades Autónomas')
data_ccaa_recovered_today = data_ccaa_recovered_today.copy().iloc[0:19]
# New cases
data_ccaa_recovered_daybefore = data_ccaa_recovered_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_recovered_daybefore.rename(columns = {date_daybefore_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_daybefore = data_ccaa_recovered_daybefore.set_index('Comunidades Autónomas')
data_ccaa_recovered_newcases = data_ccaa_recovered_today - data_ccaa_recovered_daybefore
data_ccaa_recovered_newcases = data_ccaa_recovered_newcases.copy().iloc[0:19]
data_ccaa_recovered_newcases.rename(columns = {'Curados': 'Curados (24h)'}, inplace = True)


#Juntar los tres dataframes confirmados + curados + fallecidos
data_ccaa_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_today, data_ccaa_recovered_today, data_ccaa_deaths_today,], axis=1)

# Juntar los tres dataframes nuevos casos
data_ccaa_newcases_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_newcases, data_ccaa_recovered_newcases, data_ccaa_deaths_newcases], axis=1)

# Exportar por grupos
data_ccaa_confirmed_recovered_deaths.to_csv('data_ccaa_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_newcases_confirmed_recovered_deaths.to_csv('data_ccaa_newcases_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_ccaa_newcases_confirmed_recovered_deaths

,Confirmados (24h),Curados (24h),Fallecidos (24h)
Comunidades Autónomas,,,
Andalucía,124.0,182.0,4.0
Aragón,52.0,4.0,6.0
Asturias,17.0,17.0,2.0
Baleares,20.0,0.0,4.0
C. Valenciana,66.0,90.0,10.0
Canarias,18.0,65.0,1.0
Cantabria,40.0,20.0,7.0
Castilla y León,402.0,63.0,28.0
Castilla-La Mancha,249.0,64.0,54.0


## ESPAÑA: Análisis por rangos de edad

In [13]:
#Comprobar si están subidos los últimos datos, si sale 0 es que no hay datos
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large].count()

fecha                33
rango_edad           33
sexo                 33
casos_confirmados    33
hospitalizados       33
ingresos_uci         33
fallecidos           33
dtype: int64

In [16]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('data_spain_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('data_spain_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('data_spain_ages_ambos.csv', sep = ';', encoding="utf-8")

print(data_spain_ccaa_age_ambos_concat)
print(data_spain_ccaa_age_hombres_concrete)

Confirmados  Hospitalizados  Fallecidos
Total mujeres            78747           28918        4988
Total hombres            67031           38672        7376
Total ambos sexos       145783           67595       12365
               Confirmados  Hospitalizados  Fallecidos
Rango de edad                                         
0-9                    270             110           2
10-19                  383              90           1
20-29                 2577             524          18
30-39                 5256            1471          29
40-49                 9207            3819          90
50-59                11904            6316         264
60-69                12161            8072         774
70-79                12428            9569        2254
80-89                 9926            7072        2998
90 y +                2919            1629         946


# Subir archivos a SERVIDOR desde FTP

In [18]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/
import csv
from ftplib import FTP
from io import StringIO

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]
ftp = FTP(host=host)
login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())

fp = open("data_world_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_resume.csv"), fp, 1024)

fp = open("data_world_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual.csv"), fp, 1024)


fp = open("data_world_actual_den.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual_den.csv"), fp, 1024)

fp = open("data_world_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_from100.csv"), fp, 1024)

fp = open("data_world_timeline.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_timeline.csv"), fp, 1024)

fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases.csv"), fp, 1024)

fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)

fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)


# Datos por CCAA
fp = open("data_ccaa_evolution_confirmed.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_confirmed.csv"), fp, 1024)

fp = open("data_ccaa_evolution_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_deaths.csv"), fp, 1024)

fp = open("data_ccaa_evolution_recovered.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_recovered.csv"), fp, 1024)

fp = open("data_ccaa_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_confirmed_recovered_deaths.csv"), fp, 1024)

fp = open("data_ccaa_newcases_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_newcases_confirmed_recovered_deaths.csv"), fp, 1024)

# Datos por grupos de edad
fp = open("data_spain_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_mujeres.csv"), fp, 1024)

fp = open("data_spain_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_hombres.csv"), fp, 1024)

fp = open("data_spain_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_ambos.csv"), fp, 1024)

fp.close()



230-OK. Current restricted directory is /
230 397 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 20 12:18 .
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 20 12:18 ..
-rw-------    1 akakicre   akakicre           10 Apr 21 14:31 .ftpquota
-rw-r--r--    1 akakicre   akakicre       260069 Apr  3 13:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          514 Apr 21 14:31 data_ccaa_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         5078 Apr 21 14:31 data_ccaa_evolution_confirmed.csv
-rw-r--r--    1 akakicre   akakicre         3603 Apr 21 14:31 data_ccaa_evolution_deaths.csv
-rw-r--r--    1 akakicre   akakicre         3064 Apr 21 14:31 data_ccaa_evolution_recovered.csv
-rw-r--r--    1 akakicre   akakicre          548 Apr 21 14:31 data_ccaa_newcases_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         1152 Apr 21 14:31 data_somecountries_newcases.csv
-rw-r--r--    1 akakicre   akakicre